In [3]:
#Globale paramtere for niluAPI
import sys
base_url = "https://api.nilu.no/stats/day/"
from_date = "2010-01-01"
to_date = "2024-12-31"
radius=20

In [6]:
#Spesiell funksjon som bruker generell nilu API funksjon for å hente ut data
def get_air_quality_nilu_oslo(): 
    
    #Oslo koordinater
    latitude= 59.9139
    longitude=10.7522
    
    endpoint = f"{base_url}/{from_date}/{to_date}/{latitude}/{longitude}/{radius}"

    output_file =  "../../data/raw_data/air_quality_nilu_oslo.json"

    sys.path.insert(0, '../../src')
    from niluAPI.get_data_niluAPI import fetch_air_quality_nilu
    fetch_air_quality_nilu(endpoint,  output_file)

get_air_quality_nilu_oslo()

Gruppert data er lagret under ../../data/raw_data/air_quality_nilu_oslo.json
